In [ ]:
#import nltk
#nltk.download('stopwords')

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout, Activation
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import plotly.plotly as py
#import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
#cufflinks.go_offline()
#cufflinks.set_config_file(world_readable=True, theme='pearl')

Using TensorFlow backend.


In [2]:
df1 = pd.read_csv('data_sgd/train.tsv', header=None, sep="\t")

In [3]:
df1.head()

,0,1
0,__label__1,"sounds good, i want to buy tickets."
1,__label__3,please search a hotel for me there
2,__label__4,nothing. just relax.
3,__label__5,shall i book a table here?
4,__label__6,"no, not now.i need to check my account balance?"


In [4]:
df2 = pd.read_csv('data_sgd/test.tsv', header=None, sep="\t")
df2.head()

,0,1
0,__label__23,sounds good. can you find me a rental car ther...
1,__label__26,i need to check weather on the 5th of march.
2,__label__24,cool. schedule a visit.
3,__label__14,i would like to get a cab going there.
4,__label__1,would you like some tickets?


In [5]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 80#250
# This is fixed.
EMBEDDING_DIM = 100
HIDDEN_DIM = 64


filters = 250
kernel_size = 3


tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df1[1].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 4476 unique tokens.


In [6]:
X1 = tokenizer.texts_to_sequences(df1[1].values)
X1 = pad_sequences(X1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X1.shape)

Shape of data tensor: (47022, 80)


In [7]:
Y1 = pd.get_dummies(df1[0]).values
print('Shape of label tensor:', Y1.shape)

Shape of label tensor: (47022, 21)


In [8]:
X2 = tokenizer.texts_to_sequences(df2[1].values)
X2 = pad_sequences(X2, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X2.shape)

Shape of data tensor: (9838, 80)


In [9]:
Y2 = pd.get_dummies(df2[0]).values
print('Shape of label tensor:', Y2.shape)

Shape of label tensor: (9838, 21)


In [10]:
X_train, X_test, Y_train, Y_test = X1,X2, Y1,Y2
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(47022, 80) (47022, 21)
(9838, 80) (9838, 21)


In [11]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [12]:
from keras.layers import Conv1D, GlobalMaxPooling1D

In [13]:
model = Sequential()
# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X1.shape[1]))
model.add(SpatialDropout1D(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:

model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(HIDDEN_DIM))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a softmax:
model.add(Dense(Y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m]) # metrics=['accuracy']
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 100)           5000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 80, 100)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 78, 250)           75250     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16064     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_____________________

In [14]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 42319 samples, validate on 4703 samples
Epoch 1/5





42319/42319 [==============================] - 20s 464us/step - loss: 0.8061 - acc: 0.7699 - f1_m: 0.7448 - precision_m: 0.8697 - recall_m: 0.6878 - val_loss: 0.3627 - val_acc: 0.8803 - val_f1_m: 0.8830 - val_precision_m: 0.9077 - val_recall_m: 0.8599
Epoch 2/5
42319/42319 [==============================] - 18s 421us/step - loss: 0.3249 - acc: 0.8927 - f1_m: 0.8935 - precision_m: 0.9180 - recall_m: 0.8706 - val_loss: 0.3202 - val_acc: 0.8924 - val_f1_m: 0.8954 - val_precision_m: 0.9220 - val_recall_m: 0.8707
Epoch 3/5
42319/42319 [==============================] - 18s 437us/step - loss: 0.2726 - acc: 0.9056 - f1_m: 0.9062 - precision_m: 0.9244 - recall_m: 0.8889 - val_loss: 0.3046 - val_acc: 0.8909 - val_f1_m: 0.8935 - val_precision_m: 0.9152 - val_recall_m: 0.8731
Epoch 4/5
42319/42319 [==============================] - 18s 42

In [15]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

9838/9838 [==============================] - 1s 79us/step
Test set
  Loss: 0.280
  Accuracy: 0.901


In [16]:
#loss, accuracy, f1_score, precision, recall
print(accr)

[0.28009236134887106, 0.9006911973614934, 0.9072543589364563, 0.9232265153268557, 0.8923561699653597]


In [17]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(np.argmax(Y_test, axis=1), y_pred_bool, digits=4))

9838/9838 [==============================] - 1s 63us/step
              precision    recall  f1-score   support

           0     0.8297    0.8342    0.8320       549
           1     0.9348    0.9436    0.9392       319
           2     0.9293    0.9500    0.9396       180
           3     0.9207    0.9889    0.9536       270
           4     0.9268    0.9694    0.9476       196
           5     0.8041    0.8610    0.8316       367
           6     0.8563    0.5200    0.6471       275
           7     0.9769    0.8048    0.8825       210
           8     0.9737    0.8916    0.9308       415
           9     0.9193    0.9367    0.9279       158
          10     0.7906    0.9012    0.8423       243
          11     0.9260    0.9468    0.9363       357
          12     0.7637    0.7658    0.7648       363
          13     0.8061    0.9653    0.8785       577
          14     0.8410    0.7405    0.7876       393
          15     1.0000    0.9671    0.9833       213
          16     0.9608